In [1]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexaman","key":"db5b94bbf62d6c54956a11bbf090315d"}'}

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 48.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=395ac2ecc6d4efafa9c59f4ba40a379b90876ead6bca95183756131ee10fd21c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
      .master("local")\
        .appName("imdb")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.memory", "2g") \
        .config("spark.sql.broadcastTimeout", "10h") \
        .config("spark.executor.memory", "10g") \
        .config('spark.sql.shuffle.partitions', '3') \
        .config("spark.sql.execution.arrow.enabled","true") \
        .getOrCreate()

In [6]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -q ashirwadsangwan/imdb-dataset 
!unzip /content/imdb-dataset.zip

kaggle.json
Archive:  /content/imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


In [7]:
import numpy as np
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql.functions import col ,explode,array,struct,lit
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [8]:
ratings=spark.read.csv('/content/title.ratings.tsv.gz', sep=r'\t', header=True)
castcrew1=spark.read.csv('/content/name.basics.tsv.gz', sep=r'\t', header=True)
movies=spark.read.csv('/content/title.basics.tsv', sep=r'\t', header=True)


In [9]:
castcrew1.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00373...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00599...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,soun...|tt0078723,tt00804...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0077711,tt00387...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0033870,tt00373...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [10]:
ratings.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.6|    1550|
|tt0000002|          6.1|     186|
|tt0000003|          6.5|    1207|
|tt0000004|          6.2|     113|
|tt0000005|          6.1|    1934|
|tt0000006|          5.2|     102|
|tt0000007|          5.5|     615|
|tt0000008|          5.4|    1668|
|tt0000009|          5.4|      81|
|tt0000010|          6.9|    5549|
|tt0000011|          5.2|     236|
|tt0000012|          7.4|    9440|
|tt0000013|          5.7|    1447|
|tt0000014|          7.1|    4115|
|tt0000015|          6.1|     742|
|tt0000016|          5.9|    1088|
|tt0000017|          4.7|     214|
|tt0000018|          5.4|     447|
|tt0000019|          5.5|      17|
|tt0000020|          5.0|     249|
+---------+-------------+--------+
only showing top 20 rows



In [11]:
split_col = f.split(castcrew1.knownForTitles, ',')

df = castcrew1.withColumn("mov1", split_col.getItem(0))\
    .withColumn("mov2", split_col.getItem(1))\
    .withColumn("mov3", split_col.getItem(2))\
    .withColumn("mov4", split_col.getItem(3))\
    .drop("key")

In [12]:
castcrew12=df.select('nconst','mov1','mov2','mov3','mov4')


In [13]:
castcrew12.show()

+---------+---------+---------+---------+---------+
|   nconst|     mov1|     mov2|     mov3|     mov4|
+---------+---------+---------+---------+---------+
|nm0000001|tt0050419|tt0053137|tt0043044|tt0072308|
|nm0000002|tt0117057|tt0037382|tt0071877|tt0038355|
|nm0000003|tt0049189|tt0059956|tt0054452|tt0057345|
|nm0000004|tt0078723|tt0080455|tt0072562|tt0077975|
|nm0000005|tt0050986|tt0083922|tt0069467|tt0050976|
|nm0000006|tt0077711|tt0038787|tt0036855|tt0038109|
|nm0000007|tt0033870|tt0037382|tt0034583|tt0043265|
|nm0000008|tt0078788|tt0070849|tt0068646|tt0047296|
|nm0000009|tt0059749|tt0061184|tt0057877|tt0087803|
|nm0000010|tt0031867|tt0029870|tt0035575|tt0042041|
|nm0000011|tt0035896|tt0027996|tt0034167|tt0044706|
|nm0000012|tt0031210|tt0056687|tt0035140|tt0042192|
|nm0000013|tt0053172|tt0048317|tt0060463|tt0055100|
|nm0000014|tt0031381|tt0041452|tt0040806|tt0029843|
|nm0000015|tt0048545|tt0048028|tt0044245|tt0049261|
|nm0000016|tt0091763|tt0096320|tt0057345|tt0069946|
|nm0000017|t

In [14]:
dfk= ratings.withColumn("id", col("tconst")).join(castcrew12.withColumn("id", col("mov1")), on="id")
dfk1=dfk.selectExpr("tconst","averageRating","numVotes", "nconst as one")
dfk1 = dfk1.groupBy('tconst').agg(f.concat(f.collect_list('one')))
dfk2= ratings.withColumn("id", col("tconst")).join(castcrew12.withColumn("id", col("mov2")), on="id")
dfk2=dfk2.selectExpr("tconst","nconst as two")
dfk2 = dfk2.groupBy('tconst').agg(f.concat(f.collect_list('two')))
dfk3= ratings.withColumn("id", col("tconst")).join(castcrew12.withColumn("id", col("mov3")), on="id")
dfk3=dfk3.selectExpr("tconst","nconst as three")
dfk3 = dfk3.groupBy('tconst').agg(f.concat(f.collect_list('three')))
dfk4= ratings.withColumn("id", col("tconst")).join(castcrew12.withColumn("id", col("mov4")), on="id")
dfk4=dfk4.selectExpr("tconst","nconst as four")
dfk4 = dfk4.groupBy('tconst').agg(f.concat(f.collect_list('four')))
dfk5 = dfk1.join(dfk2, on=['tconst'], how='left')
dfk6= dfk5.join(dfk3, on=['tconst'], how='left')
dfk7= dfk6.join(dfk4, on=['tconst'], how='left')
dfk8= dfk7.join(ratings, on=['tconst'], how='left')
data=dfk8.select('tconst','averageRating','numVotes',f.concat(
    f.coalesce(col("concat(collect_list(one))"), array()),
    f.coalesce(col("concat(collect_list(two))"), array()),
    f.coalesce(col("concat(collect_list(three))"), array()),
    f.coalesce(col("concat(collect_list(four))"), array())).alias("castcrew"))
data.show(truncate=False)

+---------+-------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tconst   |averageRating|numVotes|castcrew                                                                                                                                                                                                                    |
+---------+-------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tt0000003|6.5          |1207    |[nm0721526, nm5442194, nm5442200, nm5442215, nm5442293, nm1335271]                                                                                                                                    

NGRAM and Frequency Growth mining based Market Basket Analysis

Resources= 
http://repositorio.uchile.cl/bitstream/handle/2250/126856/Extending-market-basket-analysis-with-graph-mining-techniques-A-real-case.pdf?sequence=1

http://www.vldb.org/conf/1994/P487.PDF

https://www.aclweb.org/anthology/P96-1025.pdf

In [15]:
from pyspark.ml.feature import NGram
ngram = NGram(n=2, inputCol="castcrew", outputCol="bigrams")
ngramDataFrame = ngram.transform(data)

ngramDataFrame.show(truncate=False)

+---------+-------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tconst   |averageRating|numVotes|castcrew                                                                                                                                                                                                                    |bigrams                                                                                  

In [16]:
bigrams = ngramDataFrame.toPandas()['bigrams']


In [17]:

table = {}
total = len(bigrams)
completion = 0
for bigram in bigrams:
    for combination in bigram:
        components = combination.split(' ')
        key = components[0]
        valKey = components[1]
        if key in table:
            valueDict = table[key]
            if valKey in valueDict:
                valueDict[valKey] = valueDict[valKey] + 1
            else:
                valueDict[valKey] = 1
        else:
            valueDict =  {valKey: 1}
            table[key] = valueDict
    completion += 1

Showing Frequency of different cast and crew are together in more than 3 movies

In [18]:
for firstWord in table:
    for secondWord in table[firstWord]:
        if table[firstWord][secondWord] > 3:
            print(firstWord, " + ", secondWord, ": ", table[firstWord][secondWord])

nm1410831  +  nm1410832 :  4
nm1141563  +  nm1141616 :  4
nm0268716  +  nm0268717 :  4
nm3186243  +  nm3186250 :  4
nm2684503  +  nm2686766 :  4
nm0244505  +  nm0244523 :  4
nm1721924  +  nm1724662 :  4
nm0994831  +  nm0994966 :  4
nm0970179  +  nm0970239 :  4
nm0968911  +  nm0968932 :  4
nm1288519  +  nm1288938 :  4
nm2369182  +  nm2375001 :  4
nm4845589  +  nm4845617 :  4
nm0402867  +  nm0402970 :  4
nm0817915  +  nm0818139 :  4
nm4672282  +  nm4672646 :  4
nm2340048  +  nm2340183 :  4
nm1172359  +  nm1172360 :  4
nm1068656  +  nm1070798 :  4
nm2306904  +  nm2309639 :  4
nm0709120  +  nm0761694 :  4
nm5008801  +  nm5008865 :  4
nm3075069  +  nm3076505 :  4
nm1649516  +  nm1651852 :  4
nm2654523  +  nm2654526 :  4
nm2663953  +  nm2664666 :  4
nm1179503  +  nm1179582 :  4
nm1318843  +  nm1319757 :  4
nm2213735  +  nm2332106 :  4
nm1312722  +  nm1312843 :  4
nm3022868  +  nm3023013 :  4
nm2520571  +  nm2520991 :  4
nm0463004  +  nm0475491 :  4
nm1039165  +  nm1041808 :  4
nm1041977  +  

**Movie Recommendation System Logic**

For example, if in  a movie these three are together 'a', 'b', 'c'
bigrams will be 

'a b', 'b c'. 

in another movie 'b', 'c', 'd', then the bigrams will be

'b c', 'c d'.

then recommendation is given based on the bigram frequency according to the frequency dictionory created before in Market Basket Analysis
first need to sort the frequencies for each bigram in decending order.

For example,  {'a': {'c': 5, 'e':5, 'f': 7, 'g': 1}}, it will be sorted as

'a'+'f': 7

'a'+'e': 5

'a'+'c': 5

'a'+'g': 1

Actors will be recommended  from the ones with highest frequency


In [19]:
import random
def getPureData(x):
    if x not in table:
        return []
    sortedOringalList = sorted(table[x].items(), key=lambda x: x[1], reverse=True)
    data = {}
    for tp in sortedOringalList:
        product = tp[0]
        number = tp[1]
        if number in data:
            productList = data[number]
            productList.append(product)
        else:
            productList = [product]
        data[number] = productList
    pureData = data.values()
    return list(pureData)

def recommendactors(pureData, numberOfRecommend):
        
    recommendProds = []
    for prods in pureData:
        if len(prods) <= numberOfRecommend:
            recommendProds += prods
            numberOfRecommend -= len(prods)
        else:
            recommendProds += random.sample(prods, numberOfRecommend)
            numberOfRecommend = 0

        if numberOfRecommend == 0:
            break
    
    return recommendProds


def getRecommend(name, numberOfRecommend):
    
    
    
    
    recommendactorq = []
    actor = name
    index = 0

    while (numberOfRecommend):
        data = getPureData(actor)
        intermediate = recommendactors(data, numberOfRecommend)
        recommendactorq += intermediate
        if len(intermediate) == 0 and index == len(recommendactorq):
            break
        numberOfRecommend -= len(intermediate)
        if numberOfRecommend > 0:
            actor = recommendactorq[index]
            index += 1

    return recommendactorq




In [20]:
castcrew1.where(castcrew1.nconst == 'nm0000015' ).take(1)

[Row(nconst='nm0000015', primaryName='James Dean', birthYear='1931', deathYear='1955', primaryProfession='actor,miscellaneous', knownForTitles='tt0048545,tt0048028,tt0044245,tt0049261')]

In [21]:
xxx=getRecommend("nm0000015",5)
castcrew1.filter(castcrew1.nconst.isin(xxx)).select('primaryName').collect()


[Row(primaryName='Ernest Haller'),
 Row(primaryName='Ted D. McCord'),
 Row(primaryName='Adell Aldrich'),
 Row(primaryName='Mary Anderson'),
 Row(primaryName='Don Appell')]

In [22]:
movierecommendations=castcrew12.filter(castcrew12.nconst.isin(xxx))
movierecommendations=movierecommendations.select("mov1",'mov2','mov3','mov4').rdd.flatMap(lambda x: x)
movierecommendations1=movies.filter(movies.tconst.isin((movierecommendations).collect()))
movierecommendations1.select("primaryTitle").rdd.flatMap(lambda x: x).collect()


['Gone with the Wind',
 'Henry and Dizzy',
 'Lifeboat',
 'Wilson',
 'Mildred Pierce',
 'Johnny Belinda',
 'The Treasure of the Sierra Madre',
 'The Vaughn Monroe Show',
 'Campbell Summer Soundstage',
 'East of Eden',
 'Rebel Without a Cause',
 'The Vic Damone Show',
 'What Ever Happened to Baby Jane?',
 'The Sound of Music',
 'Pretty Woman',
 'Frankie and Johnny',
 'Sliver',
 'Stop! Look! and Laugh!']